In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [2]:
# partly from https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
RUG = pd.read_pickle("obfuscated_data_rm_outlier.pkl")
RUG.interpolate(method='linear', inplace=True)#, limit=20)
RUG = RUG[::10]
dfs = [RUG.filter([i]) for i in RUG]

In [5]:
dfs = [i.dropna() for i in dfs]

In [6]:
# RUG

# Timeseries LSTM

In [9]:
def func(df, name):

    df2 = df.copy()

    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(df2)

    train_size = int(len(dataset) * 0.8)
    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]


    # convert an array of values into a dataset matrix
    def create_dataset(dataset, look_back=3):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)
    
    # reshape into X=t and Y=t+1
    look_back = 3
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    
    # reshape input to be [samples, time steps, features]
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.001, verbose=2)

    # callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


    # create and fit the LSTM network
    model = Sequential()
    model.add(LSTM(4, input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    model.fit(trainX, trainY, epochs=50, verbose=2, callbacks=[early_stopping, reduce_lr], batch_size=64)

    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    # invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    # calculate root mean squared error
    rmse_train = np.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
    # print('Train Score: %.2f RMSE' % (trainScore))
    rmse_test = np.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    # print('Test Score: %.2f RMSE' % (testScore))

    mae_train = tf.keras.metrics.mean_absolute_error(trainY[0], trainPredict[:,0]).numpy()
    mae_test = tf.keras.metrics.mean_absolute_error(testY[0], testPredict[:,0]).numpy()

    mape_train = tf.keras.metrics.mean_absolute_percentage_error(trainY[0], trainPredict[:,0]).numpy()
    mape_test = tf.keras.metrics.mean_absolute_percentage_error(testY[0], testPredict[:,0]).numpy()

    return (name, (rmse_train, rmse_test, mae_train, mae_test, mape_train, mape_test))

In [11]:
results = []


In [13]:
for _ in range(3):
    for name, df in zip([i for i in RUG], dfs):
        if name not in ['Location 2 - consumption','Location 3 - consumption','Location 11 - head']:
            continue
        print(name)
        
        if df.isnull().values.any():
            print("NaN")
            results.append((name, (np.nan, np.nan)))
            print("-------------")
            continue

        r = func(df, name)
        print("-------------")
        results.append(r)

Location 2 - consumption
Epoch 1/50
3834/3834 - 13s - loss: 0.0031 - mse: 0.0031 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 2/50
3834/3834 - 13s - loss: 3.3141e-04 - mse: 3.3141e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 3/50
3834/3834 - 13s - loss: 3.2620e-04 - mse: 3.2620e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 4/50
3834/3834 - 13s - loss: 3.2604e-04 - mse: 3.2604e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 5/50
3834/3834 - 13s - loss: 3.2588e-04 - mse: 3.2588e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 6/50
3834/3834 - 13s - loss: 3.2638e-04 - mse: 3.2638e-04 - lr: 0.0010 - 13s/epoch - 4ms/step
Epoch 7/50
3834/3834 - 13s - loss: 3.2571e-04 - mse: 3.2571e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 8/50
3834/3834 - 13s - loss: 3.2669e-04 - mse: 3.2669e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 9/50
3834/3834 - 13s - loss: 3.2583e-04 - mse: 3.2583e-04 - lr: 0.0010 - 13s/epoch - 3ms/step
Epoch 10/50
3834/3834 - 13s - loss: 3.2602e-04 - mse: 3.2602e-04 - lr: 0.0010 - 13s

In [19]:
for i in results[-9:]:
    print(i)

('Location 2 - consumption', (0.2580590715162312, 0.352418768366995, 0.11972126, 0.18178856, 6467.0737, 102826.87))
('Location 3 - consumption', (0.34659605649833347, 0.5694131703751536, 0.11751799, 0.2761116, 8949769.0, 10505372.0))
('Location 11 - head', (0.43120006069999167, 0.34646928422798684, 0.23549898, 0.24129462, 0.44008476, 0.44455162))
('Location 2 - consumption', (0.26634757295218103, 0.35756471409409835, 0.14105108, 0.2002658, 4458.628, 59904.035))
('Location 3 - consumption', (0.346764782400081, 0.5693791150806413, 0.12058132, 0.2774344, 10103832.0, 11151408.0))
('Location 11 - head', (0.453748161271842, 0.3727808782851056, 0.27179107, 0.27132925, 0.50737816, 0.50023156))
('Location 2 - consumption', (0.25885531675692813, 0.3532597327783508, 0.1287216, 0.1892616, 8033.941, 136752.19))
('Location 3 - consumption', (0.35346257913252616, 0.5742139510193472, 0.1404635, 0.2907513, 5967929.0, 7558129.0))
('Location 11 - head', (0.4301075417187585, 0.34555206217023554, 0.2335708

In [25]:
lstm_results = [[i[0], i[1][0], i[1][1], i[1][2], i[1][3], i[1][4], i[1][5]] for i in results[-9:]]

lstm_results = pd.DataFrame(lstm_results)

lstm_results.columns = ["Location", "Train RMSE", "Test RMSE", "Train MAE", "Test MAE", "Train MAPE", "Test MAPE"]
lstm_results.set_index("Location", inplace=True)
lstm_results.sort_index(inplace=True)
lstm_results = lstm_results.astype(float).round(3)
lstm_results = lstm_results.groupby('Location').mean()

display(lstm_results)

# print(lstm_results.round(3).to_latex())

,Train RMSE,Test RMSE,Train MAE,Test MAE,Train MAPE,Test MAPE
Location,,,,,,
Location 11 - head,0.438333,0.355000,0.247000,0.250667,4.613333e-01,4.623333e-01
Location 2 - consumption,0.261000,0.354333,0.130000,0.190333,6.319881e+03,9.982770e+04
Location 3 - consumption,0.349000,0.570667,0.126333,0.281333,8.340510e+06,9.738303e+06


In [26]:
cols = pd.MultiIndex.from_product([['Train', 'Test'], ['RMSE', 'MAE', 'MAPE']])
lstm_results.columns = cols
print(lstm_results.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} & \multicolumn{3}{l}{Train} & \multicolumn{3}{l}{Test} \\
{} &      RMSE &       MAE &      MAPE &      RMSE &           MAE &          MAPE \\
Location                 &           &           &           &           &               &               \\
\midrule
Location 11 - head       &  0.438333 &  0.355000 &  0.247000 &  0.250667 &  4.613333e-01 &  4.623333e-01 \\
Location 2 - consumption &  0.261000 &  0.354333 &  0.130000 &  0.190333 &  6.319881e+03 &  9.982770e+04 \\
Location 3 - consumption &  0.349000 &  0.570667 &  0.126333 &  0.281333 &  8.340510e+06 &  9.738303e+06 \\
\bottomrule
\end{tabular}



C:\Users\Martin\AppData\Local\Temp\ipykernel_15588\2898845625.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(lstm_results.to_latex())
